# Set operations on RDDs

#### [Baseado em "Introduction to Spark with Python, by Jose A. Dianes"](https://github.com/jadianes/spark-py-notebooks)

O Spark suporta muitas das operações que temos em conjuntos matemáticos, como união e interseção, mesmo quando os próprios RDDs não estão configurados corretamente. É importante observar que essas operações exigem que os RDDs que estão sendo operados sejam do mesmo tipo.

Definir operações é bastante simples de entender como funciona conforme o esperado. A única consideração vem do fato de que os RDDs não são conjuntos reais e, portanto, operações como a união de RDDs não removem duplicatas. Neste notebook, vamos dar uma breve olhada em `subtract`, `distinct` e `cartesian`.

## Obtendo os dados e criando o RDD

Como fizemos em nosso primeiro notebook, usaremos o conjunto de dados reduzido 10% do fornecido para a KDD Cup 1999, contendo quase meio milhão de interações de rede. O arquivo é fornecido como um arquivo Gzip que será baixado localmente.

In [1]:
from urllib.request import urlretrieve

f = urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [2]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## Obtendo interações de ataque usando `subtract`

Para fins ilustrativos, imagine que já temos nosso RDD com interações de não ataque (normal) de alguma análise anterior.

In [5]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)

Podemos obter interações de ataque subtraindo as normais do RDD não filtrado original da seguinte maneira.

In [6]:
attack_raw_data = raw_data.subtract(normal_raw_data)

Vamos fazer algumas contagens para verificar nossos resultados.

In [9]:
from time import time

# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print("All count in {} secs".format(round(tt,3)))

All count in 3.694 secs


In [10]:
# count normal
t0 = time()
normal_raw_data_count = normal_raw_data.count()
tt = time() - t0
print("Normal count in {} secs".format(round(tt,3)))

Normal count in 3.496 secs


In [11]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print("Attack count in {} secs".format(round(tt,3)))

Attack count in 11.597 secs


In [12]:
print("There are {} normal interactions and {} attacks, \
from a total of {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count))

There are 97278 normal interactions and 396743 attacks, from a total of 494021 interactions


Então agora temos dois RDDs, um com interações normais e outro com ataques.

## Combinações de protocolos e serviços usando `cartesian`

Podemos calcular o produto cartesiano entre dois RDDs usando a transformação cartesiana. Ele retorna todos os possíveis pares de elementos entre dois RDDs. Em nosso caso, vamos usá-lo para gerar todas as combinações possíveis entre serviço e protocolo em nossas interações de rede.

Primeiro de tudo, precisamos isolar cada coleção de valores em dois RDDs separados. Para isso, usaremos `distinct` no conjunto de dados analisado por CSV. A partir da [descrição do conjunto de dados](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names) sabemos que o protocolo é a segunda coluna e o serviço é o terceiro (a tag é a última e não a primeira como aparece na página).

Então, primeiro, vamos pegar os protocolos.

In [16]:
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

['tcp', 'udp', 'icmp']

Agora fazemos o mesmo pelos serviços.

In [17]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

['http',
 'smtp',
 'finger',
 'domain_u',
 'auth',
 'telnet',
 'ftp',
 'eco_i',
 'ntp_u',
 'ecr_i',
 'other',
 'private',
 'pop_3',
 'ftp_data',
 'rje',
 'time',
 'mtp',
 'link',
 'remote_job',
 'gopher',
 'ssh',
 'name',
 'whois',
 'domain',
 'login',
 'imap4',
 'daytime',
 'ctf',
 'nntp',
 'shell',
 'IRC',
 'nnsp',
 'http_443',
 'exec',
 'printer',
 'efs',
 'courier',
 'uucp',
 'klogin',
 'kshell',
 'echo',
 'discard',
 'systat',
 'supdup',
 'iso_tsap',
 'hostnames',
 'csnet_ns',
 'pop_2',
 'sunrpc',
 'uucp_path',
 'netbios_ns',
 'netbios_ssn',
 'netbios_dgm',
 'sql_net',
 'vmnet',
 'bgp',
 'Z39_50',
 'ldap',
 'netstat',
 'urh_i',
 'X11',
 'urp_i',
 'pm_dump',
 'tftp_u',
 'tim_i',
 'red_i']

Uma lista mais longa neste caso.

Agora podemos fazer o produto cartesiano.

In [18]:
product = protocols.cartesian(services).collect()
print("There are {} combinations of protocol X service".format(len(product)))

There are 198 combinations of protocol X service


Obviamente, para esses RDDs pequenos, não faz sentido usar o produto cartesiano Spark. Poderíamos ter coletado perfeitamente os valores depois de usar `distinct` e fazer o produto cartesiano localmente. Além disso, `distinct` e `cartesian` são operações dispendiosas, portanto devem ser usadas com cuidado quando os conjuntos de dados operacionais são grandes.